### 数据挖掘基本流程

In [1]:
# 1.加载源数据处理异常值
# 2.从现有特征中提取额外特征
# 3.数据类型转换并编码
# 4.整合各表数据administr
# 5.算法建模，分割数据
# 6.多个模型训练
# 7.模型评估
# 8.参数调优
# 9.保存模型

### 1.加载源数据处理异常值

In [2]:
import pandas as pd
import numpy as np

In [3]:
#导入训练数据，测试数据和相关数据源
train = pd.read_csv('train_users_2.csv')
test = pd.read_csv('test_users.csv')
sessions1 = pd.read_csv('sessions.csv')

In [4]:
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [5]:
train.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

In [6]:
test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [7]:
test.isnull().sum()

id                             0
date_account_created           0
timestamp_first_active         0
date_first_booking         62096
gender                         0
age                        28876
signup_method                  0
signup_flow                    0
language                       0
affiliate_channel              0
affiliate_provider             0
first_affiliate_tracked       20
signup_app                     0
first_device_type              0
first_browser                  0
dtype: int64

In [8]:
sessions1.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


### train和test中 date_first_booking字段大量缺失选择删除

In [9]:
sessions = sessions1
sessions.secs_elapsed.value_counts().head()

0.0    104624
1.0     27378
2.0     23004
3.0     20375
5.0     19589
Name: secs_elapsed, dtype: int64

### sessions中的user_id可对应train中的id,先处理session数据，提取相应特征，后期再合并train

In [10]:
# 重命名user_id为id
sessions.rename(columns = {'user_id':'id'},inplace = True)

In [11]:
sessions.isnull().sum()

id                 34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

In [12]:
# action,action_type,device_type字段缺失值较多，用众数进行填充,secs_elapsed后期再进行处理
sessions.action.fillna(sessions.action.value_counts()[0],inplace = True)
sessions.action_type.fillna(sessions.action_type.value_counts()[0],inplace = True)
sessions.action_detail.fillna(sessions.action_detail.value_counts()[0],inplace = True)

sessions.secs_elapsed.fillna(0,inplace = True)
sessions.secs_elapsed.replace('other',0,inplace = True)
sessions.secs_elapsed = sessions.secs_elapsed.astype(int)

In [13]:
sessions.isnull().sum()

id               34496
action               0
action_type          0
action_detail        0
device_type          0
secs_elapsed         0
dtype: int64

In [14]:
# 将action中种类次数低于100的分为other
act_values = sessions.action.value_counts().values
def rename_act(list):
    """找出次数低于100的种类的索引并存入列表aa"""
    aa = []
    for i in range(len(list)):
        if list[i] < 100:
            aa.append(i)
    return aa
ind = rename_act(act_values)
print(ind)

# 切片出其名称索引
ind_name = sessions.action.value_counts().keys()[230:]

[230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359]


In [15]:
# %%timeit
# 14:34 -> 15:30
# 替换为other
for act_ in sessions.action:
    if act_ in ind_name:
        sessions.loc[sessions['action'] == act_] = 'other'

In [16]:
sessions.id.value_counts().head()

other         2834
mxqbh3ykxl    2722
0hjoc5q8nf    2644
mjbl6rrj52    2476
l5lgm3w5pc    2424
Name: id, dtype: int64

In [17]:
# 根据id分组
new_sessions = [new_session for new_session in sessions.groupby(sessions.id)]
len(new_sessions)

135484

In [18]:
if 'other' in sessions.secs_elapsed.value_counts().argsort().keys():
    print('true')

true


In [19]:
#将4个特征中的唯一值组合成不重复的新特征
f_act = sessions.action.value_counts().argsort()
f_act_detail = sessions.action_detail.value_counts().argsort()
f_act_type = sessions.action_type.value_counts().argsort()
f_dev_type = sessions.device_type.value_counts().argsort()

act_list = ['act_{}'.format(act) for act in range(len(f_act.keys()))]
act_detail = ['act_detail_{}'.format(act_detail) for act_detail in range(len(f_act_detail.keys()))]
act_type = ['act_type_{}'.format(act_type) for act_type in range(len(f_act_type.keys()))]
dev_list = ['dev_type_{}'.format(dev_type) for dev_type in range(len(f_dev_type.keys()))]
colunms = ['id'] + act_list + act_detail + act_type + dev_list

dict_ = dict(zip(colunms,['0']*len(colunms)))
sessions_ = pd.DataFrame(dict_,index=['0'])
sessions_

,id,act_0,act_1,act_2,act_3,act_4,act_5,act_6,act_7,act_8,...,dev_type_5,dev_type_6,dev_type_7,dev_type_8,dev_type_9,dev_type_10,dev_type_11,dev_type_12,dev_type_13,dev_type_14
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
#创建列表用于保存每个用户数据   可直接用列表生成式 name_ = ['dict_{}'.format(i) for i in range(len(new_sessions))]
name_ = ['dict_{}'.format(i) for i in range(len(new_sessions[93510:]))] 

for j in range(len(new_sessions)):   # 由于name_和待提取的特征数据列表长度一直，故直接遍历new_sessions长度
#     对secs_elapsed的缺失值填补为0，并求得停留时间的总时长，方差，平均数
    name_[j] = dict_
    sev = new_sessions[j][1].secs_elapsed.replace('other',0).values
#     print(np.log(np.sum(sev)))
    name_[j]['sev_sum'] = np.log(np.sum(sev))
#     name_[j]['sev_std'] = np.log(1 + int(np.std(sev)))
#     name_[j]['sev_mean'] = np.log(1 + int(np.mean(sev)))
    # new_sessions[0][1].secs_elapsed.value_counts()
#     print(name_[j]['sev_sum'])
    print(j)
    print('-'*30)
    print(new_sessions[93512][1].secs_elapsed.replace('other',0).values)

0
------------------------------
[0 0 0 ... 0 0 0]
1
------------------------------
[0 0 0 ... 0 0 0]
2
------------------------------
[0 0 0 ... 0 0 0]
3
------------------------------
[0 0 0 ... 0 0 0]
4
------------------------------
[0 0 0 ... 0 0 0]
5
------------------------------
[0 0 0 ... 0 0 0]
6
------------------------------
[0 0 0 ... 0 0 0]
7
------------------------------
[0 0 0 ... 0 0 0]
8
------------------------------
[0 0 0 ... 0 0 0]
9
------------------------------
[0 0 0 ... 0 0 0]
10
------------------------------
[0 0 0 ... 0 0 0]
11
------------------------------
[0 0 0 ... 0 0 0]
12
------------------------------
[0 0 0 ... 0 0 0]
13
------------------------------
[0 0 0 ... 0 0 0]
14
------------------------------
[0 0 0 ... 0 0 0]
15
------------------------------
[0 0 0 ... 0 0 0]
16
------------------------------
[0 0 0 ... 0 0 0]
17
------------------------------
[0 0 0 ... 0 0 0]
18
------------------------------
[0 0 0 ... 0 0 0]
19
-------------------

c:\users\mugglek\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


[0 0 0 ... 0 0 0]
90
------------------------------
[0 0 0 ... 0 0 0]
91
------------------------------
[0 0 0 ... 0 0 0]
92
------------------------------
[0 0 0 ... 0 0 0]
93
------------------------------
[0 0 0 ... 0 0 0]
94
------------------------------
[0 0 0 ... 0 0 0]
95
------------------------------
[0 0 0 ... 0 0 0]
96
------------------------------
[0 0 0 ... 0 0 0]
97
------------------------------
[0 0 0 ... 0 0 0]
98
------------------------------
[0 0 0 ... 0 0 0]
99
------------------------------
[0 0 0 ... 0 0 0]
100
------------------------------
[0 0 0 ... 0 0 0]
101
------------------------------
[0 0 0 ... 0 0 0]
102
------------------------------
[0 0 0 ... 0 0 0]
103
------------------------------
[0 0 0 ... 0 0 0]
104
------------------------------
[0 0 0 ... 0 0 0]
105
------------------------------
[0 0 0 ... 0 0 0]
106
------------------------------
[0 0 0 ... 0 0 0]
107
------------------------------
[0 0 0 ... 0 0 0]
108
------------------------------
[0 

[0 0 0 ... 0 0 0]
250
------------------------------
[0 0 0 ... 0 0 0]
251
------------------------------
[0 0 0 ... 0 0 0]
252
------------------------------
[0 0 0 ... 0 0 0]
253
------------------------------
[0 0 0 ... 0 0 0]
254
------------------------------
[0 0 0 ... 0 0 0]
255
------------------------------
[0 0 0 ... 0 0 0]
256
------------------------------
[0 0 0 ... 0 0 0]
257
------------------------------
[0 0 0 ... 0 0 0]
258
------------------------------
[0 0 0 ... 0 0 0]
259
------------------------------
[0 0 0 ... 0 0 0]
260
------------------------------
[0 0 0 ... 0 0 0]
261
------------------------------
[0 0 0 ... 0 0 0]
262
------------------------------
[0 0 0 ... 0 0 0]
263
------------------------------
[0 0 0 ... 0 0 0]
264
------------------------------
[0 0 0 ... 0 0 0]
265
------------------------------
[0 0 0 ... 0 0 0]
266
------------------------------
[0 0 0 ... 0 0 0]
267
------------------------------
[0 0 0 ... 0 0 0]
268
------------------------

[0 0 0 ... 0 0 0]
466
------------------------------
[0 0 0 ... 0 0 0]
467
------------------------------
[0 0 0 ... 0 0 0]
468
------------------------------
[0 0 0 ... 0 0 0]
469
------------------------------
[0 0 0 ... 0 0 0]
470
------------------------------
[0 0 0 ... 0 0 0]
471
------------------------------
[0 0 0 ... 0 0 0]
472
------------------------------
[0 0 0 ... 0 0 0]
473
------------------------------
[0 0 0 ... 0 0 0]
474
------------------------------
[0 0 0 ... 0 0 0]
475
------------------------------
[0 0 0 ... 0 0 0]
476
------------------------------
[0 0 0 ... 0 0 0]
477
------------------------------
[0 0 0 ... 0 0 0]
478
------------------------------
[0 0 0 ... 0 0 0]
479
------------------------------
[0 0 0 ... 0 0 0]
480
------------------------------
[0 0 0 ... 0 0 0]
481
------------------------------
[0 0 0 ... 0 0 0]
482
------------------------------
[0 0 0 ... 0 0 0]
483
------------------------------
[0 0 0 ... 0 0 0]
484
------------------------

...

[0 0 0 ... 0 0 0]
1188
------------------------------
[0 0 0 ... 0 0 0]
1189
------------------------------
[0 0 0 ... 0 0 0]
1190
------------------------------
[0 0 0 ... 0 0 0]
1191
------------------------------
[0 0 0 ... 0 0 0]
1192
------------------------------
[0 0 0 ... 0 0 0]
1193
------------------------------
[0 0 0 ... 0 0 0]
1194
------------------------------
[0 0 0 ... 0 0 0]
1195
------------------------------
[0 0 0 ... 0 0 0]
1196
------------------------------
[0 0 0 ... 0 0 0]
1197
------------------------------
[0 0 0 ... 0 0 0]
1198
------------------------------
[0 0 0 ... 0 0 0]
1199
------------------------------
[0 0 0 ... 0 0 0]
1200
------------------------------
[0 0 0 ... 0 0 0]
1201
------------------------------
[0 0 0 ... 0 0 0]
1202
------------------------------
[0 0 0 ... 0 0 0]
1203
------------------------------
[0 0 0 ... 0 0 0]
1204
------------------------------
[0 0 0 ... 0 0 0]
1205
------------------------------
[0 0 0 ... 0 0 0]
1206
-----

1408
------------------------------
[0 0 0 ... 0 0 0]
1409
------------------------------
[0 0 0 ... 0 0 0]
1410
------------------------------
[0 0 0 ... 0 0 0]
1411
------------------------------
[0 0 0 ... 0 0 0]
1412
------------------------------
[0 0 0 ... 0 0 0]
1413
------------------------------
[0 0 0 ... 0 0 0]
1414
------------------------------
[0 0 0 ... 0 0 0]
1415
------------------------------
[0 0 0 ... 0 0 0]
1416
------------------------------
[0 0 0 ... 0 0 0]
1417
------------------------------
[0 0 0 ... 0 0 0]
1418
------------------------------
[0 0 0 ... 0 0 0]
1419
------------------------------
[0 0 0 ... 0 0 0]
1420
------------------------------
[0 0 0 ... 0 0 0]
1421
------------------------------
[0 0 0 ... 0 0 0]
1422
------------------------------
[0 0 0 ... 0 0 0]
1423
------------------------------
[0 0 0 ... 0 0 0]
1424
------------------------------
[0 0 0 ... 0 0 0]
1425
------------------------------
[0 0 0 ... 0 0 0]
1426
-----------------------

[0 0 0 ... 0 0 0]
1571
------------------------------
[0 0 0 ... 0 0 0]
1572
------------------------------
[0 0 0 ... 0 0 0]
1573
------------------------------
[0 0 0 ... 0 0 0]
1574
------------------------------
[0 0 0 ... 0 0 0]
1575
------------------------------
[0 0 0 ... 0 0 0]
1576
------------------------------
[0 0 0 ... 0 0 0]
1577
------------------------------
[0 0 0 ... 0 0 0]
1578
------------------------------
[0 0 0 ... 0 0 0]
1579
------------------------------
[0 0 0 ... 0 0 0]
1580
------------------------------
[0 0 0 ... 0 0 0]
1581
------------------------------
[0 0 0 ... 0 0 0]
1582
------------------------------
[0 0 0 ... 0 0 0]
1583
------------------------------
[0 0 0 ... 0 0 0]
1584
------------------------------
[0 0 0 ... 0 0 0]
1585
------------------------------
[0 0 0 ... 0 0 0]
1586
------------------------------
[0 0 0 ... 0 0 0]
1587
------------------------------
[0 0 0 ... 0 0 0]
1588
------------------------------
[0 0 0 ... 0 0 0]
1589
-----

1743
------------------------------
[0 0 0 ... 0 0 0]
1744
------------------------------
[0 0 0 ... 0 0 0]
1745
------------------------------
[0 0 0 ... 0 0 0]
1746
------------------------------
[0 0 0 ... 0 0 0]
1747
------------------------------
[0 0 0 ... 0 0 0]
1748
------------------------------
[0 0 0 ... 0 0 0]
1749
------------------------------
[0 0 0 ... 0 0 0]
1750
------------------------------
[0 0 0 ... 0 0 0]
1751
------------------------------
[0 0 0 ... 0 0 0]
1752
------------------------------
[0 0 0 ... 0 0 0]
1753
------------------------------
[0 0 0 ... 0 0 0]
1754
------------------------------
[0 0 0 ... 0 0 0]
1755
------------------------------
[0 0 0 ... 0 0 0]
1756
------------------------------
[0 0 0 ... 0 0 0]
1757
------------------------------
[0 0 0 ... 0 0 0]
1758
------------------------------
[0 0 0 ... 0 0 0]
1759
------------------------------
[0 0 0 ... 0 0 0]
1760
------------------------------
[0 0 0 ... 0 0 0]
1761
-----------------------

KeyboardInterrupt: 

### 组合sessions中所有特征

In [20]:
# 找出每个用户原四个特征中的新特征唯一值并对应新特征填入其值的次数，方差，平均值
# 遍历每个按id分类的用户数据的action，通过与f_act比对，找到每个用户的action种类在f_act中的位置从而得到sessions_中的列名并插入
def get_dic():
#     dict_1 = dict_
    #创建列表用于保存每个用户数据   可直接用列表生成式 name_ = ['dict_{}'.format(i) for i in range(len(new_sessions))]
    name_ = ['dict_{}'.format(i) for i in range(len(new_sessions))]
#     for i in range(len(new_sessions)):
#         name = 'dict_{}'.format(i)
#         name_.append(name)
#     print(name_)
    list_ = []
    for j in range(len(new_sessions)):   # 由于name_和待提取的特征数据列表长度一直，故直接遍历new_sessions长度
        # 对action提取特征
        name_[j] = dict_
        inde_act = []
        for key in new_sessions[j][1].action.value_counts().keys():
            for index in range(len(f_act.keys())):
                if key == f_act.keys()[index]:
                    inde_act.append(index)

        name_[j]['id'] = new_sessions[j][0]
        for i in range(len(inde_act)):
#         print(dict_1['id'])
            name_[j]['act_{}'.format(inde_act[i])] = new_sessions[j][1].action.value_counts().values[i]
        # 对action各分类求数量，方差，平均数
        name_[j]['act_len_'] = len(new_sessions[j][1].action.value_counts().values)
        name_[j]['act_std_'] = np.std(new_sessions[j][1].action.value_counts().values)
        name_[j]['act_mean_'] = np.mean(new_sessions[j][1].action.value_counts().values)

        # 对 action_type提取特征
        inde_action_type = []
        for key in new_sessions[j][1].action_type.value_counts().keys():
            for index in range(len(f_act_type.keys())):
                if key == f_act_type.keys()[index]:
                    inde_action_type.append(index)
    #     print(inde_action_type)
        for i in range(len(inde_action_type)):
        #     print(i)
            name_[j]['act_type_{}'.format(inde_action_type[i])] = new_sessions[j][1].action_type.value_counts().values[i]
        # 对action_type各分类求数量，方差，平均数
        name_[j]['act_type_len_'] = len(new_sessions[j][1].action_type.value_counts().values)
        name_[j]['act_type_std_'] = np.std(new_sessions[j][1].action_type.value_counts().values)
        name_[j]['act_type_mean_'] = np.mean(new_sessions[j][1].action_type.value_counts().values)

        # 对 action_detail提取特征
        inde_action_detail = []
        for key in new_sessions[j][1].action_detail.value_counts().keys():
            for index in range(len(f_act_detail.keys())):
                if key == f_act_detail.keys()[index]:
                    inde_action_detail.append(index)
    #     print(inde_action_detail)
        for i in range(len(inde_action_detail)):
        #     print(i)
            name_[j]['act_detail_{}'.format(inde_action_detail[i])] = new_sessions[j][1].action_detail.value_counts().values[i]
        # 对action_detail各分类求数量，方差，平均数
        name_[j]['act_detail_len_'] = len(new_sessions[j][1].action_detail.value_counts().values)
        name_[j]['act_detail_std_'] = np.std(new_sessions[j][1].action_detail.value_counts().values)
        name_[j]['act_detail_mean_'] = np.mean(new_sessions[j][1].action_detail.value_counts().values)

        # 对 device_type提取特征
        inde_device_type = []
        for key in new_sessions[j][1].device_type.value_counts().keys():
            for index in range(len(f_dev_type.keys())):
                if key == f_dev_type.keys()[index]:
                    inde_device_type.append(index)
    #     print(inde_device_type)
        for i in range(len(inde_device_type)):
        #     print(i)
            name_[j]['dev_type_{}'.format(inde_device_type[i])] = new_sessions[j][1].device_type.value_counts().values[i]
        # 对device_type各分类求数量，方差，平均数
        name_[j]['dev_type_len_'] = len(new_sessions[j][1].device_type.value_counts().values)
        name_[j]['dev_type_std_'] = np.std(new_sessions[j][1].device_type.value_counts().values)
        name_[j]['dev_type_mean_'] = np.mean(new_sessions[j][1].device_type.value_counts().values)

        # 对secs_elapsed的缺失值填补为0，并求得停留时间的总时长，方差，平均数
        sev = new_sessions[j][1].secs_elapsed.replace('other',0).values
        name_[j]['sev_sum'] = np.log(1 + int(np.sum(sev)))
        name_[j]['sev_std'] = np.log(1 + int(np.std(sev)))
        name_[j]['sev_mean'] = np.log(1 + int(np.mean(sev)))
        # new_sessions[0][1].secs_elapsed.value_counts()
        yield name_[j]

In [21]:
get_dic()

<generator object get_dic at 0x000001D6044A52B0>

In [22]:
results = get_dic()

In [23]:
# %%timeit   魔术方法不能保存cell结果
# 10:48 -> 12:37  2H
# 将用id分好组的数据转为dataframe并保存为列表
list2_ = []
for x in results:
    new_columns = colunms+['act_len_','act_std_','act_mean_','act_type_len_','act_type_std_','act_type_mean_','act_detail_len_','act_detail_std_','act_detail_mean_','dev_type_len_','dev_type_std_','dev_type_mean_','sev_sum','sev_std','sev_mean']
    df_seesion = pd.DataFrame(x,columns = new_columns,index = ['0'])
    list2_.append(df_seesion)

In [29]:
len(list2_)

135484

In [31]:
# %%timeit
# 合并dataframe并重置索引
new_df = list2_[0]
for j in range(len(list2_[:10000])):
    new_df = pd.concat([new_df,list2_[j]],axis = 0)  #迭代
new_df.reset_index(drop = True,inplace = True)
new_df.drop(index = [0],axis = 0,inplace = True)
new_df.head()

,id,act_0,act_1,act_2,act_3,act_4,act_5,act_6,act_7,act_8,...,act_type_mean_,act_detail_len_,act_detail_std_,act_detail_mean_,dev_type_len_,dev_type_std_,dev_type_mean_,sev_sum,sev_std,sev_mean
1,00023iyk9l,9,4,1,4,0,2,0,3,0,...,5.714286,14,1.641304,2.857143,2,16.0,20.0,13.673828,11.407287,9.984975
2,0010k6l0om,20,5,8,8,0,8,0,3,0,...,10.500000,9,4.853407,7.000000,1,0.0,63.0,13.282003,10.017664,9.138952
3,001wyh0pz8,6,2,8,8,66,8,1,3,2,...,15.000000,10,19.073542,9.000000,1,0.0,90.0,12.553082,8.780326,8.053569
4,0028jgx1x1,15,2,8,8,9,8,1,3,2,...,6.200000,6,3.287180,5.166667,2,14.5,15.5,12.601524,9.770128,9.167537
5,002qnbzfs5,232,115,8,8,125,8,13,3,72,...,98.625000,23,47.020287,34.304348,2,380.5,394.5,15.685323,10.968853,9.014569


In [32]:
new_df.sev_sum.head()

1    13.673828
2    13.282003
3    12.553082
4    12.601524
5    15.685323
Name: sev_sum, dtype: float64

In [33]:
new_df.to_csv('sessions.csv')

### 挖掘新特征

In [ ]:
# 对action各分类求数量，方差，平均数
act_len_ = len(new_sessions[0][1].action.value_counts().values)
act_std_ = np.std(new_sessions[0][1].action.value_counts().values)
act_mean_ = np.mean(new_sessions[0][1].action.value_counts().values)
# 对action_type各分类求数量，方差，平均数
act_type_len_ = len(new_sessions[0][1].action_type.value_counts().values)
act_type_std_ = np.std(new_sessions[0][1].action_type.value_counts().values)
act_type_mean_ = np.mean(new_sessions[0][1].action_type.value_counts().values)
# 对action_detail各分类求数量，方差，平均数
act_detail_len_ = len(new_sessions[0][1].action_detail.value_counts().values)
act_detail_std_ = np.std(new_sessions[0][1].action_detail.value_counts().values)
act_detail_mean_ = np.mean(new_sessions[0][1].action_detail.value_counts().values)
# 对device_type各分类求数量，方差，平均数
dev_type_len_ = len(new_sessions[0][1].device_type.value_counts().values)
dev_type_std_ = np.std(new_sessions[0][1].device_type.value_counts().values)
dev_type_mean_ = np.mean(new_sessions[0][1].device_type.value_counts().values)

In [ ]:
# 对secs_elapsed的缺失值填补为，并求得停留时间的总时长，方差，平均数
sev = new_sessions[0][1].secs_elapsed.fillna(0).values
sev_sum = np.log(1 + np.sum(sev))
sev_std = np.log(1 + np.std(sev))
sev_mean = np.log(1 + np.mean(sev))
# new_sessions[0][1].secs_elapsed.value_counts()

### 处理train和test数据

In [ ]:
# 获取train的最大行数，为后面分割做准备
train_rows = train.shape[0]
train_rows

In [ ]:
# 提取标签（预测值）
labels = train['country_destination']
labels.head()

In [ ]:
# 获取train特征值并合并test，一起处理
train_cat = train.iloc[:,:train.shape[1]-1]
data = pd.concat([train_cat,test],axis = 0,ignore_index = True)
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# 由于date_first_booking字段缺失值较多，所以删除特征
data.drop(['date_first_booking'],axis = 1,inplace = True)
data.head()

In [ ]:
# 用age和first_affiliate_tracked的众数填补缺失值
data.age.fillna(data.age.value_counts().argsort().values[0],inplace = True)
data.first_affiliate_tracked.fillna(data.first_affiliate_tracked.value_counts().argsort().values[0],inplace = True)
data.isnull().sum()

In [ ]:
# age中有些是19**或20**，故判断为出生年份，由于数据是14年统计，所以将其转为年龄；将age限定区间
age = data.age.values
aa = []
for a in age:
    if a >= 1900:
        a = 2014 - a
    aa.append(a)
data['age'] = aa
def get_age(age):
    if age < 15:
        return 15
    elif age < 30:
        return 22
    elif age < 45:
        return 35
    elif age < 60:
        return 55
    elif age < 75:
        return 67
    elif age <= 90:
        return 84
    else:
        return 100
bb = []
for b in data.age.values:
    bb.append(get_age(b))
data['age'] = bb

In [ ]:
# timestamp_first_active转为datetime
import datetime
data.timestamp_first_active = data.timestamp_first_active.astype(str).apply(lambda x:  
                                                                    datetime.datetime(int(x[:4]),
                                                                                      int(x[4:6]), 
                                                                                      int(x[6:8]), 
                                                                                      int(x[8:10]), 
                                                                                      int(x[10:12]),
                                                                                      int(x[12:])))
# 从timestamp_first_active中提取年月日，一周哪一天，四大特征
data['tfa_year'] = [tfa_year.year for tfa_year in data.timestamp_first_active]
data['tfa_month'] = [tfa_month.month for tfa_month in data.timestamp_first_active]
data['tfa_day'] = [tfa_day.day for tfa_day in data.timestamp_first_active]
data['tfa_weekday'] = [tfa_weekday.isoweekday() for tfa_weekday in data.timestamp_first_active]
# 将提取的tfa_weekday编码
df_tfa_weekday = pd.get_dummies(data.tfa_weekday , prefix='tfa_weekday')
data = pd.concat([data,df_tfa_weekday],axis = 1)
data.drop(['tfa_weekday'],axis = 1,inplace = True)

In [ ]:
# 转换date_account_created格式
dac = pd.to_datetime(data.date_account_created)
# 提取新特征：用户在airbnb平台活跃到正式注册所花的时间；timestamp_first_active与date_account_created的差值
df_span = dac.subtract(data.timestamp_first_active).dt.days
def get_span(dt):
    # dt is an integer
    if dt == -1:
        return 'OneDay'
    elif (dt < 30) & (dt > -1):
        return 'OneMonth'
    elif (dt >= 30) & (dt <= 365):
        return 'OneYear'
    else:
        return 'other'

data['dt_span'] = np.array([get_span(x) for x in df_span])
df_dt_span = pd.get_dummies(data.dt_span, prefix = 'dt_span')
data = pd.concat((data, df_dt_span), axis = 1)
data.drop(['dt_span'], axis = 1, inplace = True)

In [ ]:
# 从date_account_created中提取年月日，一周哪一天，四大特征
data['dac_year'] = [dac_year.year for dac_year in dac]
data['dac_month'] = [dac_month.month for dac_month in dac]
data['dac_day'] = [dac_day.day for dac_day in dac]
data['dac_weekday'] = [dac_weekday.isoweekday() for dac_weekday in dac]

df_tfa_weekday = pd.get_dummies(data.dac_weekday , prefix='tfa_weekday')
data = pd.concat([data,df_tfa_weekday],axis = 1)
data.drop(['dac_weekday'],axis = 1,inplace = True)
# 删除不必要的特征
data.drop(['date_account_created','timestamp_first_active'],axis = 1,inplace = True)

In [ ]:
# 将其余的特征one_hot编码
feat_toOHE = ['gender', 
             'signup_method', 
             'signup_flow', 
             'language', 
             'affiliate_channel', 
             'affiliate_provider', 
             'first_affiliate_tracked', 
             'signup_app', 
             'first_device_type', 
             'first_browser']
#对其他特征进行one-hot-encoding处理
for f in feat_toOHE:
    df_ohe = pd.get_dummies(data[f], prefix=f, dummy_na=True)
    data.drop([f], axis = 1, inplace = True)
    data = pd.concat((data, df_ohe), axis = 1)

In [ ]:
# 标记预测值
from sklearn.preprocessing import LabelEncoder
l_e = LabelEncoder()
encode_label = l_e.fit_transform(labels.values)
encode_label

In [ ]:
data.to_csv('data.csv')